In [1]:
### Atom class + readframe + lines_in_file functions - used to process dump files

import numpy as np
import operator # do we need this?

class Atom:
    """ A Class for storing atom information """

    def __init__(self):
        """ Initialise the class """
        self.id = 0                                              # id of the atom
        self.type = 0                                            # type of the atom
        self.L = np.array([0.0,0.0,0.0],dtype=np.float64)        # size of simulation box
        self.half_L = self.L / 2                                 # half the size of simulation box
        self.x = np.array([0.0,0.0,0.0],dtype=np.float64)        # position of the atom
        self.image = np.array([0,0,0],dtype=np.int32)            # image flags for atoms
        self.x_unwrap = np.array([0.0,0.0,0.0],dtype=np.float64) # position of the atom - unwrapped coords
        self.unwrap_flag = False

    
    def sep(self, atom2):
        """ Takes in self and atom2 and finds separation between them taking into account periodic BCs """

        dx = self.x[0] - atom2.x[0]
        dy = self.x[1] - atom2.x[1]
        dz = self.x[2] - atom2.x[2]

        if dx > self.half_L[0]:
            dx = self.L[0] - dx
        if dy > self.half_L[1]:
            dy = self.L[1] - dy
        if dz > self.half_L[2]:
            dz = self.L[2] - dz

        return np.sqrt(dx**2 + dy**2 + dz**2)
    
    def sep_2(self, atom2):
        """ Takes in self and atom2 and finds their separation SQUARED WITHOUT taking into account periodic BCs """

        dx = self.x[0] - atom2.x[0]
        dy = self.x[1] - atom2.x[1]
        dz = self.x[2] - atom2.x[2]

        """
        if dx > self.half_L[0]:
            dx = self.L[0] - dx
        if dy > self.half_L[1]:
            dy = self.L[1] - dy
        if dz > self.half_L[2]:
            dz = self.L[2] - dz
        """

        return dx**2 + dy**2 + dz**2

    def minus(self,B):
        """ Subtract B.x vector from self.x vector """
        
        AminusB = np.array([0.0,0.0,0.0],dtype=np.float64)
        for i in range(3):
            AminusB[i] = self.x[i] - B.x[i]
            
        return AminusB

    def xdot(self,B):
        """ Find dot product of position x of this Atom and Atom B """
        
        AdotB = np.array([0.0,0.0,0.0],dtype=np.float64) # TO DO : find AdotB ??
        
        return AdotB

    # confused as to what this function does :/ - isn't unwrap_flag always False?
    def unwrap(self):
        """ Unwraps the coordinates for periodic box to generate x_unwrap array """
        
        if not self.unwrap_flag:   # first check it has not already been done
            for j in range(3):
                self.x_unwrap[j] = self.x[j] + self.image[j]*self.L[j] # unwrap
            unwrap_flag = True

In [2]:
### functions

def readframe(infile, N):
    """ Read a single frame of N atoms from a dump file 
        Expects coordinates to be in range -L/2 to L/2
        DOES NOT Unwrap corrdinates for periodic box """

    atoms = [Atom() for i in range(N)]
    L = np.array([0.0,0.0,0.0],dtype=np.float64)

    # read in the 9 header lines of the dump file
    # get box size
    for i in range(9):
        line = infile.readline()
        if i==1:  ## second line of frame is timestep
            timestep = np.int32(line)
        if i==5 or i==6 or i==7:   # 6th-8th lines are box size in x,y,z dimensions
            # get the box size
            line = line.split()
            L[i-5]=np.float64(line[1]) - np.float64(line[0]);

    # now read the atoms, putting them at the correct index (index=id-1)
    for i in range(N):
        line = infile.readline()
        line = line.split()
        index = int(line[0])-1  # LAMMPS atom ids start from 1, python index starts from 0
        atoms[index].id = int(line[0])
        atoms[index].type = int(line[1])
        atoms[index].L = L
        for j in range(3):
            atoms[index].x[j] = np.float64(line[j+2])
        for j in range(3):
            atoms[index].image[j] = np.int32(line[j+5])

    return atoms, timestep


def lines_in_file(filename):
    """ Get the number of lines in the file """

    with open(filename) as f:
        for i, l in enumerate(f):
            pass

    return i + 1

In [3]:
### calculation functions

import statistics as s

def dbscan(atoms, threshold, target_type):
    """ Takes in a list of Atom objects, distance threshold + target type. 
        Sets the cluster id for atoms we are not interested in to -2.
        Returns a list of cluster ids where the ith element is the cluster id for ith atom in input list """

    if not atoms: # edge case of empty atom list - return an empty cluster id list
        return []

    cluster_id = 0
    cluster_ids = [-1] * len(atoms)  # Initialize cluster IDs for each atom; -1 means unclassified

    threshold_2 = threshold**2

    def find_neighbors(atom_index):
        """ Takes in index of an atom and find the neighbours of that atom. 
            Atoms are neighbours if within threshold distance of 2.4.
            Returns a list of neighbours of type=4 for atom index inputted """

        return [i for i, other_atom in enumerate(atoms)
                if i != atom_index and atoms[atom_index].type == target_type and other_atom.type == target_type and 
                atoms[atom_index].sep_2(other_atom) < threshold_2]
    
    # loops through atoms list
    for i in range(len(atoms)):

        # checks to see if cluster id of ith atom is -1 - if not atom already processed
        if cluster_ids[i] != -1 or atoms[i].type != target_type:

            # checks to see if atom type is target type - if not sets cluster id of that atom to -2 (not of interest)
            if atoms[i].type != target_type:
                cluster_ids[i] = -2

            continue

        neighbors = find_neighbors(i) # finds neighbours of atom i

        cluster_id += 1
        cluster_ids[i] = cluster_id

        k = 0

        # for every neighbour of atom i, finds the neighbours of that neighbour and adds it to original neighbour list
        while k < len(neighbors):
            neighbor_idx = neighbors[k]
            if cluster_ids[neighbor_idx] == -1:
                cluster_ids[neighbor_idx] = cluster_id

                new_neighbors = find_neighbors(neighbor_idx)
                for new_neighbor in new_neighbors:
                    if cluster_ids[new_neighbor] == -1:
                        neighbors.append(new_neighbor)
            k += 1
            
    no_of_clusters = cluster_id # cluster_id acts like a 'cluster count'

    return no_of_clusters, cluster_ids

In [4]:
def size_of_clusters(cluster_ids):
    """ Takes in a list of cluster ids. Returns a list of the number of proteins in a cluster, ignoring cluster ids of -2.
        Eg: [-2, -2, ..., 1, 4, 2, 4, 1, 3, 4, ...] --> 2 in cluster 1, 1 in cluster 2, 1 in cluster 3, 3 in cluster 4 ... """
    
    size_of_clusters = [0] * max(cluster_ids) # 0 means 0 atoms in cluster i
    
    # loop through list of cluster ids
    for i in cluster_ids:
        
        # if the cluster id is not -2 (-2 are atoms of non-interest)
        if i != -2:
            # increase value of i-1 by 1
            size_of_clusters[i-1] += 1
                    
    return size_of_clusters



def mean_size_of_clusters(size_of_clusters):
    """ Takes in list of cluster sizes (in number of proteins) and returns the mean number of proteins in a cluster """
    
    return s.fmean(size_of_clusters) # fmean runs faster than mean apparently



def size_of_largest_cluster(size_of_clusters):
    """ Takes in list of cluster sizes and returns the number of proteins in largest cluster """
    
    return max(size_of_clusters)


def no_of_clusters_size_1(size_of_clusters):
    """ Takes in a list of cluster sizes and returns the number of clusters with only 1 protein - i.e. size 1 """

    return size_of_clusters.count(1)


def no_proteins_bound_to_poly(atoms, target_types={1, 2, 3}, threshold_2=3.24):
    """ Takes in a list of Atom objects. 
        Returns the no of proteins bound to the polymer of type=1, 2 or 3 + list of no of polymer beads an atom is bound to. """

    def find_polymers_bound_to(j):
        """ Takes in index of a protein (type 4) and finds if it is bound to polymer (type=1, 2 or 3). 
            Bound to polymer if within threshold distance of 1.8. Or if sep_2 < threshold_2 (3.24).
            Returns a list of indices of polymer beads of type=1, 2 or 3 for atom index inputted. 
            Length of list is no of polymer beads that protein is bound to. """

        return [i for i, other_atom in enumerate(atoms)
                #if i != j and other_atom.type in {2} and atoms[j].sep2(other_atom) < threshold_2]
                if i != j and other_atom.type in target_types and atoms[j].sep_2(other_atom) < threshold_2]


    no_proteins_bound = 0 # intialises counter of number of proteins bound to a polymer bead to 0
    no_polymers_bound_to = [0] * len(atoms) # initialises list of number of polymer beads that each protein is bound to, to 0

    # loops through all atoms - could make it loop through just the last 300 as those are the proteins!
    for j, atom in enumerate(atoms):

        # if atom is a protein (type 4)
        if atom.type == 4:

            # find number of polymer beads (of any type - 1, 2 or 3) that protein is bound to
            poly_beads_list = find_polymers_bound_to(j)
            no_polymers_bound_to[j] = len(poly_beads_list) # length of list is the no of polymer beads that protein is bound to

            # if list is not empty
            if poly_beads_list:
                no_proteins_bound += 1

    return no_proteins_bound, no_polymers_bound_to

In [5]:
def fraction_clusters_bound_to_poly(atoms, cluster_ids, cluster_no):
    """ Takes in a list of Atom objects, their cluster ids and no_of_clusters (at each frame). 
        Returns the fraction of clusters bound to the polymer of type=1, 2 or 3 """

    def protein_bound_to_poly(j):
        """ Takes in index of a protein in cluster (type 4) and finds if it is bound to polymer (type=1, 2 or 3). 
            Bound to polymer if within threshold distance of 1.8. Or if sep_2 < threshold_2 (3.24). 
            Stops and returns True if it is bound. False if not bound. """
        for i, other_atom in enumerate(atoms):
            if i != j and other_atom.type in target_types and atoms[j].sep_2(other_atom) < threshold_2:
                return True # stop when you find a protein in cluster that is within threshold distance of any polymer bead
        return False

    bound_clusters = []  # list of cluster ids of bound clusters
    print("bound clusters list: ", bound_clusters)
    target_types={1, 2, 3}
    threshold_2=3.24

    # loop through all cluster ids
    for j, cluster_id in enumerate(cluster_ids):
        
        # if cluster_id is not -2 (ie: it is the cluster id of a protein) and it is not already in bound_clusters
        if cluster_id != -2 and cluster_id not in bound_clusters:

            # check if the protein belonging to that cluster id is bound to a polymer bead. If it is add cluster id to bound_clusters list.
            if protein_bound_to_poly(j):
                bound_clusters.append(cluster_id)

    fraction_bound = len(bound_clusters) / cluster_no # no of bound clusters / total no of clusters
    
    print("---------------------------------------------")
    print("total no of clusters: ",cluster_no)
    print("bound clusters list: ", bound_clusters)
    print("no of bound clusters: ",len(bound_clusters))
    print("frac of bound clusters: ",fraction_bound)
    print("---------------------------------------------")

    return fraction_bound

In [6]:
def mean_no_type_2_poly_in_cluster(atoms, cluster_ids, target_poly_types={2}, threshold_2=3.24):
    """ Takes in a list of Atom objects and cluster_ids.
        Returns the mean number of type 2 polymers bound to proteins in a cluster """

    def find_type_2_polymers_bound_to(j):
        """ Takes in index of a protein (type 4) and finds if it is bound to polymer (type=2). 
            Bound to polymer if within threshold distance of 1.8. Or if sep_2 < threshold_2 (3.24).
            Returns a list of indices of polymer beads of type=2 for protein index inputted. 
            Length of list is no of polymer beads that protein is bound to. """

        return [i for i, other_atom in enumerate(atoms)
                if i != j and other_atom.type in target_poly_types and atoms[j].sep_2(other_atom) < threshold_2]

    no_type_2_poly_in = {} # dictionary of cluster ids to no of type 2 polymers in that cluster id

    # loop through all cluster ids
    for j, cluster_id in enumerate(cluster_ids):
        
        # if cluster_id is not -2 (ie: it is the cluster id of a protein)
        if cluster_id != -2 and cluster_id not in no_type_2_poly_in:

            # find the atom indices of type 2 polymer beads that protein is bound to
            poly_beads_list = find_type_2_polymers_bound_to(j)

            # find the rest of the proteins in the cluster that the protein is in
            for i, cluster_id_i in enumerate(cluster_ids):

                if cluster_id_i == cluster_id and i != j:
                    
                    # add to list of indices of type 2 polymers bound to proteins in cluster_id
                    poly_beads_list += find_type_2_polymers_bound_to(i) # note: there will be repeated indices in the list
        
            # remove duplicates from list + add no of type 2 poly beads in that cluster to dictionary
            no_type_2_poly_in[cluster_id] = len(set(poly_beads_list))

    return s.fmean(no_type_2_poly_in.values())

In [10]:
from tqdm import tqdm

### main programs --> need to change this cause very sucky!

######################## - User inputs (sys argv command line) but worse cause need to remember the order of inputs :/

"""
dumpfilename = sys.argv[1] # name of dump file to read - dump.sticky_DNA+proteins

Natoms = int(sys.argv[2]) # no of atoms - 220
Npoly = int(sys.argv[3]) # no of polymer atoms - 200

outfile_Rg = sys.argv[4] # name of output file - r_g_sticky_DNA

thresh = float(sys.argv[5]) # cluster threshold - 2.4
"""

######################## - Hardcoded (use for testing)

name_dumpfile = 'dump_model_1_run_1.dat' # name of dump file to read - dump.sticky_DNA+proteins

n_atoms = 5300 # no of atoms - 220
n_poly_atoms = 5000 # no of polymer atoms - 200

name_outfile = 'outfile_test_1.dat' # name of output file - r_g_sticky_DNA

######################## - User inputs (command line) - best way imo
"""
name_dumpfile = input("Name of dumpfile: ")

n_atoms = int(input("Integer no of atoms (in df): "))
n_poly_atoms = int(input("Integer no of polymer atoms (in df): "))

name_outfile = input("Name of output file: ")
"""
########################

# TO DO: could make this more automatic by giving a default of everything... user can say y/n to default and change if required

threshold = 2.4 # cluster threshold - 2.4
target_type = 4  # target atom type - should change this to type 4 for model simulations... / could add as user input

path_to_dumpfiles = '../../dumpfiles/' # need to change this when running on local computer
path_to_outfiles = '../outfiles/'

n_lines = lines_in_file(path_to_dumpfiles + name_dumpfile) # no of lines in file
n_frames = int(n_lines / (n_atoms + 9)) # +9 as 9 header lines in each frame

# open the input file
file_in = open(path_to_dumpfiles + name_dumpfile, 'r')

# open the output file and print a header
file_out = open(path_to_outfiles + name_outfile, 'w')
file_out.write("# Timesteps, Mean no of type 2 atoms per cluster\n")

# go through the file frame by frame
for frame in tqdm(range(n_frames)):
    # read the frame, unwrapping periodic coordinates
    atoms, timesteps = readframe(file_in, n_atoms)
    
    # unwarp period boundary coordinates -- needed for clusters? - think so as a cluster can form between boundaries
    for i in range(len(atoms)):
        atoms[i].unwrap()

    # perform calculations
    no_of_clusters, cluster_ids = dbscan(atoms, threshold, target_type)
    #cluster_size = size_of_clusters(cluster_ids)
    #mean_cluster_size = mean_size_of_clusters(cluster_size)
    #largest_cluster_size = size_of_largest_cluster(cluster_size)
    
    #no_proteins_bound, no_polymers_bound_to = no_proteins_bound_to_poly(atoms)
    
    #frac_clusters_bound = fraction_clusters_bound_to_poly(atoms, cluster_ids, int(no_of_clusters))

    mean_no = mean_no_type_2_poly_in_cluster(atoms, cluster_ids)
    print(mean_no)
    
    # output some results
    #file_out.write("%i %i %i %i\n"%(timesteps, no_of_clusters, mean_cluster_size, largest_cluster_size))
    #file_out.write("%i %i %.2f\n"%(timesteps, no_of_clusters, frac_clusters_bound))
    file_out.write("%i %.2f\n"%(timesteps, mean_no))

# close the files
file_in.close()
file_out.close()

  0%|▎                                                                                                                        | 1/401 [00:00<03:41,  1.81it/s]

0.017985611510791366


  0%|▌                                                                                                                        | 2/401 [00:01<04:12,  1.58it/s]

0.233201581027668


  1%|▉                                                                                                                        | 3/401 [00:01<04:22,  1.51it/s]

...

0.5857142857142857


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 400/401 [03:49<00:00,  1.80it/s]

0.6044776119402985


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 401/401 [03:49<00:00,  1.75it/s]

0.5673758865248227


In [ ]:
import os
from tqdm import tqdm  # Ensure you have tqdm installed

# Assuming necessary functions are defined elsewhere:
# lines_in_file, readframe, dbscan, size_of_clusters, mean_size_of_clusters,
# size_of_largest_cluster, no_of_clusters_size_1, no_proteins_bound_to_poly

threshold = 2.4  # cluster threshold
target_type = 4  # target atom type

n_atoms = 5300 #int(input("Integer no of atoms (in df): "))
n_poly_atoms = 5000

path_to_dumpfiles = '../../lammps_sims/dumpfiles/'
path_to_outfiles = '../outfiles/'

# Assuming 'name_dumpfile' and 'n_atoms' are defined elsewhere
name_dumpfile = 'dump_model_4_var_1_run_1.dat'  # Replace with actual dumpfile name
name_outfile = 'outfile_test.dat'            # Replace with desired outfile name
name_cluster_size_file = 'cluster_sizes_test.dat'  # New file for cluster sizes

n_lines = lines_in_file(os.path.join(path_to_dumpfiles, name_dumpfile))
n_frames = int(n_lines / (n_atoms + 9))  # +9 for header lines

# Open the input file
file_in = open(os.path.join(path_to_dumpfiles, name_dumpfile), 'r')

# Open the output file and print a header
file_out = open(os.path.join(path_to_outfiles, name_outfile), 'w')
file_out.write("# Timesteps, No of clusters, Mean cluster size, Size largest cluster, No clusters size 1, No proteins bound to poly\n")

# Open the new file for cluster sizes
file_cluster_size_out = open(os.path.join(path_to_outfiles, name_cluster_size_file), 'w')

# Process the file frame by frame
for frame in tqdm(range(n_frames)):
    # Read the frame, unwrapping periodic coordinates
    atoms, timesteps = readframe(file_in, n_atoms)
    
    # Unwrap periodic boundary coordinates
    for i in range(len(atoms)):
        atoms[i].unwrap()

    # Perform calculations on clusters
    no_of_clusters, cluster_ids = dbscan(atoms, threshold, target_type)
    cluster_size = size_of_clusters(cluster_ids)
    print("cluster sizes: ",cluster_size)
    mean_cluster_size = mean_size_of_clusters(cluster_size)
    #largest_cluster_size = size_of_largest_cluster(cluster_size)
    #size_1_count = no_of_clusters_size_1(cluster_size)
    #no_proteins_bound, no_polymers_bound_to = no_proteins_bound_to_poly(atoms)
    
    # Output results to the main file
    file_out.write("%i %i %.5f\n" % (timesteps, no_of_clusters, mean_cluster_size))

    # Output the cluster_size list to the cluster size file
    file_cluster_size_out.write(f"{timesteps}: {cluster_size}\n")

# Close all files
file_in.close()
file_out.close()
file_cluster_size_out.close()

In [ ]:
from collections import Counter
import matplotlib.pyplot as plt

# Initialize a Counter to keep track of cluster sizes
cluster_size_counter = Counter()

with open("/home/s2205640/Documents/summer_project/mod_lammps_code/outfiles/cluster_sizes_test.dat", "r") as file:
    for line in file:
        # Extract the cluster sizes part after the colon and strip any surrounding whitespace
        _, sizes_str = line.split(":")
        sizes_str = sizes_str.strip()  # Remove any leading/trailing whitespace
        
        # Evaluate the string representation of the list into an actual list object of integers
        sizes_list = eval(sizes_str)
        
        # Update the counter with the sizes list
        cluster_size_counter.update(sizes_list)

# Print the counts for debugging purposes
print(cluster_size_counter)

In [ ]:
# Extract sizes and their counts from the counter
sizes = list(cluster_size_counter.keys())
counts = list(cluster_size_counter.values())

sizes_list_step_1 = (np.arange(0, max(sizes), 1)).tolist()
sizes_list_step_10 = (np.arange(0, max(sizes), 5)).tolist()

# gets value for key if in cluster_size_counter (ie. gets counts), if not it defaults that count to 0
counts_list = [cluster_size_counter.get(size, 0) for size in sizes_list]

# Create a bar plot
plt.figure(figsize=(16, 10))
plt.bar(sizes_list, counts_list, color='skyblue')
plt.xlabel('Cluster Size')
plt.ylabel('Frequency')
plt.title('Distribution of Cluster Sizes')
plt.xticks(sizes_list_step_10)  # Ensure each cluster size is a tick on the x-axis
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()